In [4]:
import shutil
import csv
import os
import sys
import traceback
import random
import pickle
import enum
import copy

import numpy as np

import torch

from absl import flags
from absl import app
from tqdm import tqdm

from pysc2.lib import point
from pysc2.lib import features as F
from pysc2.lib import actions as A
from pysc2.lib.actions import RAW_FUNCTIONS
from pysc2.lib import units as Unit
from pysc2 import run_configs
from pysc2.env.sc2_env import SC2Env, AgentInterfaceFormat

from s2clientprotocol import sc2api_pb2 as sc_pb
from s2clientprotocol import common_pb2 as com_pb

from alphastarmini.core.arch.agent import Agent
from alphastarmini.core.sl.feature import Feature
from alphastarmini.core.sl.label import Label

from alphastarmini.lib.hyper_parameters import Arch_Hyper_Parameters as AHP
from alphastarmini.lib.hyper_parameters import AlphaStar_Agent_Interface_Format_Params as AAIFP
from alphastarmini.lib.hyper_parameters import Label_Size as LS

from alphastarmini.lib import utils as U

In [11]:
class Config:
    def __init__(self):
        # Game rendering settings
        self.render = False
        self.realtime = False
        self.full_screen = False
        
        # Game performance settings
        self.fps = 22.4
        self.step_mul = 1
        self.render_sync = False
        self.screen_resolution = 64
        self.minimap_resolution = 64  # Assuming AHP.minimap_size was 64
        
        # Game steps limits
        self.max_game_steps = 0
        self.max_episode_steps = 0
        self.max_replays = 100
        self.max_steps_of_replay = int(22.4 * 60 * 60)  # 1 hour of game
        self.small_max_steps_of_replay = 256
        
        # Thresholds
        self.no_op_threshold = 0.000
        self.move_camera_threshold = 0.00
        self.Smart_pt_threshold = 0.0
        self.Smart_unit_threshold = 0.1
        self.Harvest_Gather_unit_threshold = 0.1
        self.Attack_pt_threshold = 0.5
        
        # Game settings
        self.disable_fog = False
        self.observed_player = 1
        
        # Save settings
        self.save_type = 0  # 0: torch_tensor, 1: python_pickle, 2: numpy_array
        self.replay_version = "3.16.1"
        
        # Paths
        self.replay_path = "/home/tc/git_repo/mls/explore_mas/data_simple64/"
        self.save_path = "./data/replay_data/"
        self.save_path_tensor = "/home/tc/git_repo/mls/explore_mas/tr/"
        self.no_server_replay_path = self.replay_path
        
        # Additional parameters from param.py
        self.on_server = False
        self.Batch_Scale = 16
        self.Seq_Scale = 16
        self.Select_Scale = 4
        self.handle_cuda_error = False
        self.skip_entity_list = False
        self.skip_autoregressive_embedding = False

# Create instances
FLAGS = Config()
P = Config()  # If you need both FLAGS and P to have the same configuration

# You can modify any settings easily:
# FLAGS.replay_path = "your/new/path"
# P.Batch_Scale = 32

In [12]:
RACE = ['Terran', 'Zerg', 'Protoss', 'Random']
RESULT = ['Victory', 'Defeat', 'Tie']

SIMPLE_TEST = not P.on_server
if SIMPLE_TEST:
    DATA_FROM = 0
    DATA_NUM = 2
else:
    DATA_FROM = 0
    DATA_NUM = 90
SMALL_MAX_STEPS = 1000  # 5000, 60 * 60 * 22.4


class SaveType(enum.IntEnum):
    torch_tensor = 0
    python_pickle = 1
    numpy_array = 2


if FLAGS.save_type == 0:
    SAVE_TYPE = SaveType.torch_tensor
elif FLAGS.save_type == 1:
    SAVE_TYPE = SaveType.python_pickle
else:
    SAVE_TYPE = SaveType.numpy_array


def check_info(replay_info):
    map_name = replay_info.map_name
    player1_race = replay_info.player_info[0].player_info.race_actual
    player2_race = replay_info.player_info[1].player_info.race_actual

    print('map_name:', map_name)
    print('player1_race:', player1_race)
    print('player2_race:', player2_race)

    return True


def store_info(replay_info):
    map_name = replay_info.map_name
    player1_race = RACE[replay_info.player_info[0].player_info.race_requested - 1]
    player2_race = RACE[replay_info.player_info[1].player_info.race_requested - 1]
    game_duration_loops = replay_info.game_duration_loops
    game_duration_seconds = replay_info.game_duration_seconds
    game_version = replay_info.game_version
    game_result = RESULT[replay_info.player_info[0].player_result.result - 1]
    return [map_name,
            game_version,
            game_result,
            player1_race,
            player2_race,
            game_duration_loops,
            game_duration_seconds]


def getFeatureAndLabel_numpy(obs, func_call, delay=None, last_list=None, build_order=None):
    print("begin s:") if debug else None

    s = Agent.get_state_and_action_from_pickle_numpy(obs, build_order=build_order, last_list=last_list)

    feature = Feature.state2feature_numpy(s)
    print("feature:", feature) if debug else None
    print("feature.shape:", feature.shape) if debug else None

    print("begin a:") if debug else None
    print('func_call', func_call) if debug else None
    action = Agent.func_call_to_action(func_call)
    action.delay = min(delay, LS.delay_encoding - 1)

    print("action:", action) if debug else None
    action_array = action.toArray()
    print("action_array:", action_array) if debug else None

    a = action_array.toLogits_numpy()
    print("a:", a) if debug else None
    label = Label.action2label_numpy(a)
    print("label:", label) if debug else None
    print("label.shape:", label.shape) if debug else None

    return feature, label


def getObsAndFunc(obs, func_call, z):
    # add z
    [bo, bu] = z

    # for build order has nothing, we make it be a list has only one item:
    # if len(bo) == 0:
    #    bo = [0]

    bo = np.array(bo)
    bu = np.array(bu)

    print("bo", bo) if debug else None
    print("bu", bu) if debug else None

    # Extract build order and build units vectors from replay
    # bo = replay.get_BO(replay.home_player)
    # bu = replay.get_BU(replay.home_player)

    BO_PROBABILITY = 0.8
    BU_PROBABILITY = 0.5

    # Sample Boolean variables bool_BO and bool_BU
    # bool_bo = np.float(np.random.rand(*bo.shape) < BO_PROBABILITY)
    # bool_bu = np.float(np.random.rand(*bu.shape) < BU_PROBABILITY)

    bool_bo = np.random.rand(*bo.shape) < BO_PROBABILITY
    bool_bu = np.random.rand(*bu.shape) < BU_PROBABILITY

    bool_bo = bool_bo.astype('float32')
    bool_bu = bool_bu.astype('float32')

    print("bool_bo", bool_bo) if debug else None
    print("bool_bu", bool_bu) if debug else None

    # Generate masked build order and build units
    masked_bo = bool_bo * bo
    masked_bu = bool_bu * bu

    print("masked_bo", masked_bo) if debug else None
    print("masked_bu", masked_bu) if debug else None
    print("sum(masked_bu)", sum(masked_bu)) if debug else None

    last_actions = obs["last_actions"]
    upgrades = obs["upgrades"]
    unit_counts = obs["unit_counts"] 
    feature_effects = obs["feature_effects"]
    raw_effects = obs["raw_effects"]

    feature_minimap = obs["feature_minimap"]

    height_map = feature_minimap["height_map"]
    camera = feature_minimap["camera"]
    visibility_map = feature_minimap["visibility_map"]
    creep = feature_minimap["creep"]
    player_relative = feature_minimap["player_relative"]
    alerts = feature_minimap["alerts"]
    pathable = feature_minimap["pathable"]
    buildable = feature_minimap["buildable"]

    step_dict = {'raw_units': obs["raw_units"][:AHP.max_entities],                 
                 'player': obs["player"],

                 'last_actions': last_actions,
                 'upgrades': upgrades,
                 'unit_counts': unit_counts,
                 'feature_effects': feature_effects,
                 'raw_effects': raw_effects,

                 'height_map': height_map,
                 'camera': camera,
                 'visibility_map': visibility_map,
                 'creep': creep,
                 'player_relative': player_relative,
                 'alerts': alerts,
                 'pathable': pathable,
                 'buildable': buildable,

                 'game_loop': obs["game_loop"],

                 'masked_bo': masked_bo, 
                 'masked_bu': masked_bu,

                 'func_call': func_call}

    return step_dict


def getFuncCall(o, feat, obs, use_raw=True):
    func_call = None

    if use_raw:
        # note: o.actions can be larger than 1, (when the replays are generated by 
        # the control of agent and the human player at the sametime, e.g., in the 
        # DAGGER method process),
        # at that case, the first action in o.actions may always be no-op!
        # so only use o.actions[0] can get nothing information!
        # we should iterater over all action in o.actions!
        print('len(o.actions): ', len(o.actions)) if debug else None

        action_list = []
        for action in o.actions:
            # becareful, though here recommand using prev_obs,
            # we should use obs instead. If not it will get the unit index of the last obs, not this one! 
            # not used: raw_func_call = feat.reverse_raw_action(o.actions[0], prev_obs)
            raw_func_call = feat.reverse_raw_action(action, obs)

            if raw_func_call.function.value == 0:
                # no op
                pass
            elif raw_func_call.function.value == 168:
                # camera move
                pass
            elif raw_func_call.function.value == 1:
                # we do not consider the smart point
                pass        
            else:
                print('raw_func_call: ', raw_func_call) if debug else None
                action_list.append(raw_func_call)

        if len(action_list) > 0:
            func_call = action_list
    else:

        raise NotImplementedError

    return func_call


def test(on_server=False):

    if on_server:
        REPLAY_PATH = P.replay_path  # "/home/liuruoze/data4/mini-AlphaStar/data/filtered_replays_1/" 
        COPY_PATH = None
        SAVE_PATH = "./result.csv"
        max_steps_of_replay = FLAGS.max_steps_of_replay
        max_replays = FLAGS.max_replays
    else:
        REPLAY_PATH = FLAGS.no_server_replay_path
        COPY_PATH = None
        SAVE_PATH = "./result.csv"
        max_steps_of_replay = SMALL_MAX_STEPS  # 60 * 60 * 22.4  # 60 * 60 * 22.4
        max_replays = 1  # not used

    run_config = run_configs.get(version=FLAGS.replay_version)
    print('REPLAY_PATH:', REPLAY_PATH)
    replay_files = os.listdir(REPLAY_PATH)
    print('length of replay_files:', len(replay_files))
    replay_files.sort()

    screen_resolution = point.Point(FLAGS.screen_resolution, FLAGS.screen_resolution)
    minimap_resolution = point.Point(FLAGS.minimap_resolution, FLAGS.minimap_resolution)
    camera_width = 24

    # By default raw actions select, act and revert the selection. This is useful
    # if you're playing simultaneously with the agent so it doesn't steal your
    # selection. This inflates APM (due to deselect) and makes the actions hard
    # to follow in a replay. Setting this to true will cause raw actions to do
    # select, act, but not revert the selection.
    raw_affects_selection = False

    # Changes the coordinates in raw.proto to be relative to the playable area.
    # The map_size and playable_area will be the diagonal of the real playable area.
    crop_to_playable_area = False
    raw_crop_to_playable_area = False

    interface = sc_pb.InterfaceOptions(
        raw=True, 
        score=True,
        # Omit to disable.
        feature_layer=sc_pb.SpatialCameraSetup(width=camera_width),  
        # Omit to disable.
        render=None,  
        # By default cloaked units are completely hidden. This shows some details.
        show_cloaked=False, 
        # By default burrowed units are completely hidden. This shows some details for those that produce a shadow.
        show_burrowed_shadows=False,       
        # Return placeholder units (buildings to be constructed), both for raw and feature layers.
        show_placeholders=False, 
        # see below
        raw_affects_selection=raw_affects_selection,
        # see below
        raw_crop_to_playable_area=raw_crop_to_playable_area
    )

    screen_resolution.assign_to(interface.feature_layer.resolution)
    minimap_resolution.assign_to(interface.feature_layer.minimap_resolution)
    interface.feature_layer.crop_to_playable_area = crop_to_playable_area

    agent = Agent()
    #j = 0
    replay_length_list = []
    noop_length_list = []

    from_index = DATA_FROM
    end_index = DATA_FROM + DATA_NUM

    with run_config.start(full_screen=False) as controller:

        for i, replay_file in enumerate(tqdm(replay_files)):
            try:
                replay_path = REPLAY_PATH + replay_file
                print('replay_path:', replay_path)

                do_write = False
                if i >= from_index:
                    if end_index is None:
                        do_write = True
                    elif end_index is not None and i < end_index:
                        do_write = True

                if not do_write:
                    continue 

                replay_data = run_config.replay_data(replay_path)
                replay_info = controller.replay_info(replay_data)

                print('replay_info', replay_info) if debug else None
                print('type(replay_info)', type(replay_info)) if debug else None

                print('replay_info.player_info：', replay_info.player_info) if debug else None
                infos = replay_info.player_info

                observe_id_list = []
                observe_result_list = []
                for info in infos:
                    print('info：', info) if debug else None
                    player_info = info.player_info
                    result = info.player_result.result
                    print('player_info', player_info) if debug else None
                    if player_info.race_actual == com_pb.Protoss:
                        observe_id_list.append(player_info.player_id)
                        observe_result_list.append(result)

                print('observe_id_list', observe_id_list) if debug else None
                print('observe_result_list', observe_result_list) if debug else None

                win_observe_id = 0

                for i, result in enumerate(observe_result_list):
                    if result == sc_pb.Victory:
                        win_observe_id = observe_id_list[i]
                        break

                # we observe the winning one
                print('win_observe_id', win_observe_id)

                if win_observe_id == 0:
                    print('no win_observe_id found! continue')
                    continue

                start_replay = sc_pb.RequestStartReplay(
                    replay_data=replay_data,
                    options=interface,
                    disable_fog=False,  # FLAGS.disable_fog
                    observed_player_id=win_observe_id,  # random.randint(1, 2),  # 1 or 2, wo random select it. FLAGS.observed_player
                    map_data=None,
                    realtime=False
                )

                print(" Replay info ".center(60, "-")) if 1 else None
                print("replay_info", replay_info) if 1 else None
                print("-" * 60) if debug else None
                controller.start_replay(start_replay)
                # The below several arguments are default set to False, so we shall enable them.

                # use_feature_units: Whether to include feature_unit observations.

                # use_raw_units: Whether to include raw unit data in observations. This
                # differs from feature_units because it includes units outside the
                # screen and hidden units, and because unit positions are given in
                # terms of world units instead of screen units.

                # use_raw_actions: [bool] Whether to use raw actions as the interface.
                # Same as specifying action_space=ActionSpace.RAW.

                # use_unit_counts: Whether to include unit_counts observation. Disabled by
                # default since it gives information outside the visible area. 

                '''
                show_cloaked: Whether to show limited information for cloaked units.
                show_burrowed_shadows: Whether to show limited information for burrowed
                      units that leave a shadow on the ground (ie widow mines and moving
                      roaches and infestors).
                show_placeholders: Whether to show buildings that are queued for
                      construction.
                '''

                aif = AgentInterfaceFormat(**AAIFP._asdict())

                feat = F.features_from_game_info(game_info=controller.game_info(),
                                                 raw_resolution=AAIFP.raw_resolution, 
                                                 crop_to_playable_area=crop_to_playable_area,
                                                 raw_crop_to_playable_area=raw_crop_to_playable_area,
                                                 hide_specific_actions=AAIFP.hide_specific_actions,
                                                 use_feature_units=True, use_raw_units=True,
                                                 use_unit_counts=True, use_raw_actions=True,
                                                 show_cloaked=True, show_burrowed_shadows=True, 
                                                 show_placeholders=True) 

                # consistent with the SL and RL setting
                # feat = F.features_from_game_info(game_info=controller.game_info(), agent_interface_format=aif)

                print("feat obs spec:", feat.observation_spec()) if debug else None
                print("feat action spec:", feat.action_spec()) if debug else None
                prev_obs = None
                prev_function = None
                i = 0
                record_i = 0
                save_steps = 0
                noop_count = 0
                obs_list, func_call_list, z_list, delay_list = [], [], [], [] 
                feature_list, label_list = [], []
                step_dict = {}

                # initial build order
                player_bo = []
                player_ucb = []
                bo_list = []

                while True:
                    o = controller.observe()
                    try:
                        obs = feat.transform_obs(o)

                        try:
                            func_call = None
                            no_op = False
                            if o.actions:
                                function_calls = getFuncCall(o, feat, obs)

                                if function_calls is not None:
                                    # when remove no_op and other actions
                                    # we can only reserver for the only the first one action
                                    func_call = function_calls[0]

                                    if func_call.function.value == 0:
                                        no_op = True
                                        func_call = None
                                    elif func_call.function.value == RAW_FUNCTIONS.raw_move_camera.id.value:  # raw_move_camera
                                        if random.uniform(0, 1) > FLAGS.move_camera_threshold:
                                            func_call = None
                                    elif func_call.function.value == RAW_FUNCTIONS.Smart_pt.id.value:  # Smart_pt
                                        if random.uniform(0, 1) > FLAGS.Smart_pt_threshold:
                                            func_call = None
                                    elif func_call.function.value == RAW_FUNCTIONS.Smart_unit.id.value:  # Smart_unit
                                        if random.uniform(0, 1) > FLAGS.Smart_unit_threshold:
                                            func_call = None
                                    elif func_call.function.value == RAW_FUNCTIONS.Harvest_Gather_unit.id.value:  # Harvest_Gather_unit
                                        if random.uniform(0, 1) > FLAGS.Harvest_Gather_unit_threshold:
                                            func_call = None
                                    elif func_call.function.value == RAW_FUNCTIONS.Attack_pt.id.value:  # Harvest_Gather_unit
                                        if random.uniform(0, 1) > FLAGS.Attack_pt_threshold:
                                            func_call = None
                            else:
                                no_op = True

                            if no_op:
                                print('expert func: no op') if debug else None
                                if random.uniform(0, 1) < FLAGS.no_op_threshold:
                                    print('get no op !') if debug else None
                                    noop_count += 1
                                    func_call = A.FunctionCall.init_with_validation("no_op", [], raw=True)

                            if func_call is not None:
                                save_steps += 1
                                #z = [player_bo, player_ucb]

                                delay = i - record_i
                                print('two action dealy:', delay, 'steps!') if debug else None
                                record_i = i

                                obs_list.append(obs)

                                func_call_list.append(func_call)
                                print('func_call:', func_call) if 1 else None

                                delay_list.append(delay)

                                if prev_obs is not None:
                                    # calculate the build order
                                    player_bo = U.calculate_build_order(player_bo, prev_obs, obs)

                                    player_bo_show = [U.get_unit_tpye_name_and_race(U.get_unit_tpye_from_index(index))[0].name for index in player_bo]
                                    print("player build order:", player_bo_show) if debug else None

                                bo_list.append(copy.deepcopy(player_bo))
                                prev_obs = obs

                        except Exception as e:
                            traceback.print_exc()

                        if i >= max_steps_of_replay:  # test the first n frames
                            print("max frames test, break out!")
                            break

                        if o.player_result:  # end of game
                            print('o.player_result', o.player_result)
                            break

                    except Exception as inst:
                        traceback.print_exc() 

                    controller.step()

                    i += 1

                print('begin save!')

                # the last delay is 0
                delay_list.append(0)
                func_call_list.append(A.FunctionCall.init_with_validation("no_op", [], raw=True))

                if SAVE_TYPE == SaveType.torch_tensor:
                    length = len(obs_list)
                    for i in range(length):
                        obs = obs_list[i]
                        func_call = func_call_list[i]
                        bo = bo_list[i]

                        # delay should be the next,
                        # means when should we issue the next command from the time we give this command
                        last_delay = delay_list[i]
                        last_func_call = func_call_list[i - 1]
                        last_action_type = last_func_call.function.value
                        action_can_be_queued = U.action_can_be_queued(last_action_type)
                        last_repeat_queued = None
                        last_action_arguments = None
                        if action_can_be_queued:
                            last_action_arguments = last_func_call.arguments
                            last_repeat_queued = last_action_arguments[0][0].value  # the first argument is alway queue
                        else:
                            last_repeat_queued = 0

                        delay = delay_list[i + 1]

                        print('last_func_call', last_func_call) if debug else None
                        print('last_action_type', last_action_type) if debug else None
                        print('last_action_can_be_queued', action_can_be_queued) if debug else None
                        print('last_action_arguments', last_action_arguments) if debug else None
                        print('last_repeat_queued', last_repeat_queued) if debug else None
                        print('last_delay', last_delay) if debug else None

                        print('func_call', func_call) if debug else None
                        print('delay', delay) if debug else None

                        last_list = [last_delay, last_action_type, last_repeat_queued]

                        feature, label = getFeatureAndLabel_numpy(obs, func_call, delay, last_list, bo)

                        feature = torch.tensor(feature)
                        label = torch.tensor(label)
                        feature_list.append(feature)
                        label_list.append(label)

                elif SAVE_TYPE == SaveType.python_pickle:
                    length = len(obs_list)
                    for i in range(length):
                        obs = obs_list[i]
                        func_call = func_call_list[i]
                        delay = delay_list[i + 1]
                        the_dict = getObsAndFunc(obs, func_call, z)
                        step_dict[i] = the_dict

                elif SAVE_TYPE == SaveType.numpy_array:
                    pass

                if SAVE_TYPE == SaveType.torch_tensor:
                    features = torch.cat(feature_list, dim=0)
                    labels = torch.cat(label_list, dim=0)
                    print('features.shape:', features.shape) if debug else None
                    print('labels.shape:', labels.shape) if debug else None
                    #m = {'features': features, 'labels': labels}
                    m = (features, labels)

                    if not os.path.exists(FLAGS.save_path_tensor):
                        os.mkdir(FLAGS.save_path_tensor)
                    file_name = FLAGS.save_path_tensor + replay_file.replace('.SC2Replay', '') + '.pt'
                    torch.save(m, file_name)

                elif SAVE_TYPE == SaveType.python_pickle:
                    file_name = FLAGS.save_path + replay_file.replace('.SC2Replay', '') + '.pickle'
                    with open(file_name, 'wb') as handle:
                        pickle.dump(step_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

                elif SAVE_TYPE == SaveType.numpy_array:
                    pass    

                print('end save!')

                replay_length_list.append(save_steps)
                noop_length_list.append(noop_count)
                # We only test the first one replay            
            except Exception as inst:
                traceback.print_exc() 

            # not used
            # if j >= max_replays:  # test the first n frames
            #     print("max replays test, break out!")
            #     break

    print("end")
    print("replay_length_list:", replay_length_list)
    print("noop_length_list:", noop_length_list)

In [20]:
import os
from pathlib import Path

def clean_duplicate_files(source_dir, target_dir):
    """
    Compares filenames (ignoring extensions) between source_dir and target_dir.
    Deletes files from source_dir if their base name exists in target_dir.
    
    Args:
        source_dir: Path to source directory (/home/tc/git_repo/mls/explore_mas/data_simple64)
        target_dir: Path to target directory (/home/tc/git_repo/mls/explore_mas/tr)
    """
    # Convert paths to Path objects
    source_path = Path(source_dir)
    target_path = Path(target_dir)
    
    # Get all filenames without extensions from both directories
    target_names = {f.stem for f in target_path.iterdir() if f.is_file()}
    source_files = [f for f in source_path.iterdir() if f.is_file()]
    
    # Keep track of deleted files
    deleted_files = []
    
    # Check each file in source directory
    for source_file in source_files:
        if source_file.stem in target_names:
            os.remove(source_file)
            deleted_files.append(source_file.name)
    
    # Print summary
    print(f"Deleted {len(deleted_files)} files from {source_dir}")
    if deleted_files:
        print("Deleted files:")
        for file in deleted_files:
            print(f"  - {file}")
    else:
        print("No files were deleted.")

# Example usage
source_directory = "/home/tc/git_repo/mls/explore_mas/data_simple64"
target_directory = "/home/tc/git_repo/mls/explore_mas/tr"

In [22]:
import os
from pathlib import Path

directory = "/home/tc/git_repo/mls/explore_mas/data_simple64"

while True:
    files = [f for f in Path(directory).iterdir() if f.is_file()]
    if not files:
        break
    
    # Your code here
    test()
    clean_duplicate_files(source_directory, target_directory)
    print(f"Files remaining: {len(files)}")

REPLAY_PATH: /home/tc/git_repo/mls/explore_mas/data_simple64/
length of replay_files: 88


Version: B55958 (SC2.3.16)
Build: Jul 31 2017 13:19:41
Command Line: '"/home/tc/StarCraftII/Versions/Base55958/SC2_x64" -listen 127.0.0.1 -port 36913 -dataDir /home/tc/StarCraftII/ -tempDir /tmp/sc-degrx8rj/ -dataVersion 5BD7C31B44525DAB46E64C4602A81DC2 -eglpath libEGL.so'
Starting up...
Startup Phase 1 complete
Startup Phase 2 complete
Creating stub renderer...
Listening on: 127.0.0.1:36913 (36913)
Startup Phase 3 complete. Ready for commands.
  0%|          | 0/88 [00:00<?, ?it/s]Could not find map name for file: /tmp/sc-degrx8rj/StarCraft II/TempReplayInfo.SC2Replay
Configuring interface options
Configure: raw interface enabled
Configure: feature layer interface enabled
Configure: score interface enabled
Configure: render interface disabled
Launching next game.
Next launch phase started: 2
Next launch phase started: 3
Next launch phase started: 4
Next launch phase started: 5
Next launch phase started: 6
Next launch phase started: 7
Next launch phase started: 8


replay_path: /home/tc/git_repo/mls/explore_mas/data_simple64/Simple64_2021-11-21-12-05-26.SC2Replay
win_observe_id 1
----------------------- Replay info ------------------------
replay_info map_name: "Simple Test Map 64x64"
local_map_path: "Melee/Simple64.SC2Map"
player_info {
  player_info {
    player_id: 1
    race_requested: Protoss
    race_actual: Protoss
  }
  player_result {
    player_id: 1
    result: Victory
  }
  player_apm: 23
}
player_info {
  player_info {
    player_id: 2
    race_requested: Terran
    race_actual: Terran
  }
  player_result {
    player_id: 2
    result: Defeat
  }
  player_apm: 37
}
game_duration_loops: 9396
game_duration_seconds: 419.4935607910156
game_version: "3.16.1.55958"
data_build: 55958
base_build: 55958
data_version: "5BD7C31B44525DAB46E64C4602A81DC2"



Starting replay 'TempStartReplay.SC2Replay'
Game has started.
Deferred error: 536936448 (0x20010000)
Deferred error details: Bad profiling data. TimeMS: 7302917.000000 Conversion: 1.000000e-03


func_call: FunctionCall(function=<_Raw_Functions.Train_Probe_quick: 64>, arguments=[[<Queued.now: 0>], [49]])
func_call: FunctionCall(function=<_Raw_Functions.Train_Probe_quick: 64>, arguments=[[<Queued.now: 0>], [54]])
func_call: FunctionCall(function=<_Raw_Functions.Build_Pylon_pt: 35>, arguments=[[<Queued.now: 0>], [51], [16, 26]])
func_call: FunctionCall(function=<_Raw_Functions.Build_Pylon_pt: 35>, arguments=[[<Queued.now: 0>], [45], [21, 21]])
func_call: FunctionCall(function=<_Raw_Functions.Train_Probe_quick: 64>, arguments=[[<Queued.now: 0>], [54]])
func_call: FunctionCall(function=<_Raw_Functions.Train_Probe_quick: 64>, arguments=[[<Queued.now: 0>], [48]])


...

Could not find map name for file: /tmp/sc-ds_oltuz/StarCraft II/TempReplayInfo.SC2Replay
Configuring interface options
Configure: raw interface enabled
Configure: feature layer interface enabled
Configure: score interface enabled
Configure: render interface disabled
Launching next game.
Next launch phase started: 2
Next launch phase started: 3
Next launch phase started: 4
Next launch phase started: 5
Next launch phase started: 6
Next launch phase started: 7
Next launch phase started: 8
Starting replay 'TempStartReplay.SC2Replay'
Game has started.


func_call: FunctionCall(function=<_Raw_Functions.Train_Probe_quick: 64>, arguments=[[<Queued.now: 0>], [52]])
func_call: FunctionCall(function=<_Raw_Functions.Build_Pylon_pt: 35>, arguments=[[<Queued.now: 0>], [55], [21, 26]])
func_call: FunctionCall(function=<_Raw_Functions.Smart_unit: 12>, arguments=[[<Queued.queued: 1>], [56], [38]])
func_call: FunctionCall(function=<_Raw_Functions.Build_Gateway_pt: 37>, arguments=[[<Queued.now: 0>], [47], [19, 27]])


100%|██████████| 2/2 [00:18<00:00,  9.10s/it]

max frames test, break out!
begin save!
end save!
end
replay_length_list: [3, 4]
noop_length_list: [0, 0]
Deleted 2 files from /home/tc/git_repo/mls/explore_mas/data_simple64
Deleted files:
  - Simple64_2021-12-28-07-47-43.SC2Replay
  - Simple64_2021-12-28-07-45-45.SC2Replay
Files remaining: 2



RequestQuit command received.
Closing Application...
